版权所有 2025 Google LLC。

In [ ]:
#@title 根据 Apache 许可证 2.0 版（“许可证”）授权；
# 除非遵守许可证，否则您不得使用此文件。
# 您可以通过以下网址获取许可证副本：
#
# https://www.apache.org/licenses/LICENSE-2.0  
#
# 除非适用法律要求或书面同意，软件
# 根据许可证分发是基于“原样”基础，
# 不提供任何形式的担保或条件。
# 请参阅许可证以了解特定语言下的权限和
# 限制。

<table align="left">
  <td>
      <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Convert_Gemma_3_270M_to_LiteRT_for_MediaPipe_LLM_Inference_API.ipynb  "><img src="https://www.tensorflow.org/images/colab_logo_32px.png  " />在 Google Colab 中运行</a>
  </td>
</table>

# 将 Gemma 3 270M 转换为 LiteRT 以用于 MediaPipe LLM 推理 API

本笔记本将 Gemma 3 270M 模型转换为适用于 [MediaPipe LLM 推理 API](https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference) 的格式，该库支持在移动设备或网页浏览器中运行推理。整个过程大约需要 15 分钟：

1. 设置 Colab 环境
2. 从 Hugging Face 加载模型
3. 使用 AI Edge Torch 转换器转换模型
4. 使用 MediaPipe Task 打包器打包模型
5. 下载模型

Gemma 3 270M 专为特定任务微调而设计，并针对移动设备、网页和边缘设备的高效性能进行了优化。你可以使用[这个笔记本](https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Fine_tune_Gemma_3_270M_for_emoji_generation.ipynb)微调自己的模型，并在转换后运行在[网页演示应用](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-mediapipe)中。

## 设置开发环境

第一步是使用 pip 安装所需的包。

In [ ]:
%pip uninstall -y tensorflow
%pip install -U tf-nightly==2.21.0.dev20250819 ai-edge-torch==0.6.0 protobuf transformers
%pip install -U jax jaxlib

重启会话运行时以确保使用新安装的包。

## 加载模型
要访问 Hugging Face 上的模型，请提供你的[访问令牌](https://huggingface.co/settings/tokens)。你可以将其添加到环境变量变量名称为 `HF_TOKEN`，值为你的令牌。

In [ ]:
import os
from huggingface_hub import login
login(hf_token)

指定要转换的模型的 Hugging Face 仓库 ID。它将保存到你的 Colab 文件中以供转换。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_author = ""                                         #@param {type:"string"}
model_name = "myemoji-gemma-3-270m-it"                    #@param {type:"string"}

repo_id = f"{model_author}/{model_name}"                  # 要转换的模型
save_path = f"/content/{model_name}"                      # 保存调整后模型的路径

model = AutoModelForCausalLM.from_pretrained(repo_id)     # 加载模型
tokenizer = AutoTokenizer.from_pretrained(repo_id)        # 加载分词器

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"模型和分词器已保存到 {save_path}")

## 转换模型
使用 [AI Edge Torch](https://github.com/google-ai-edge/ai-edge-torch) 转换器转换并量化模型。你可以根据任务需求调整转换参数：

* `prefill_seq_len`：支持的最大输入长度
* `kv_cache_max_len`：prefill + decode 上下文的最大长度
* `quantize`：量化方案。8 位整数量化（INT8）适用于网页环境

这个过程大约需要 10 分钟。.tflite 模型将临时保存到你的 Colab 文件中。

In [ ]:
from ai_edge_torch.generative.examples.gemma3 import gemma3
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities.export_config import ExportConfig
from ai_edge_torch.generative.layers import kv_cache

# 获取模型，设置导出配置，并转换为 .tflite
pytorch_model = gemma3.build_model_270m(save_path)
export_config = ExportConfig()
export_config.kvcache_layout = kv_cache.KV_LAYOUT_TRANSPOSED
export_config.mask_as_input = True
converter.convert_to_tflite(
    pytorch_model,
    output_path="/content",
    output_name_prefix=model_name,
    prefill_seq_len=128,
    kv_cache_max_len=512,
    quantize="dynamic_int8",
    export_config=export_config,
)

print (f"模型已转换为 .tflite 并保存到 {save_path}")

## 创建 MediaPipe Task 包

MediaPipe Task 文件（.task）打包了原始模型分词器、LiteRT 模型（.tflite）以及使用 MediaPipe LLM 推理 API 运行端到端推理所需的附加元数据。

为了使用打包器，在这一步安装 MediaPipe PyPI 包（>0.10.14），因为它有自己的依赖项。

In [ ]:
%pip install mediapipe

重新安装 `protobuf` 和 `tensorflow` 并重启 Colab 运行时以获取最新版本。

In [ ]:
%pip uninstall protobuf -y && pip install protobuf
%pip uninstall tensorflow -y -q && pip install tensorflow

现在，你将配置并创建 Task 包：

1. 更新 `tflite_model` 指向你刚刚转换的 .tflite 模型
2. 更新 `tokenizer_model` 指向从 Hugging Face Hub 下载的 tokenizer.model 文件
3. 在 `output_filename` 中命名你的 .task 文件

In [ ]:
from mediapipe.tasks.python.genai import bundler

config = bundler.BundleConfig(
    tflite_model="/content/myemoji-gemma-3-270m-it_q8_ekv512.tflite",     # 指向你转换的 .tflite 模型
    tokenizer_model="/content/myemoji-gemma-3-270m-it/tokenizer.model",   # 指向下载的 tokenizer.model 文件
    start_token="<bos>",
    stop_tokens=["<eos>", "<end_of_turn>"],
    output_filename="/content/myemoji-gemma-3-270m-it.task",              # 指定最终模型文件名
    prompt_prefix="<start_of_turn>user\n",
    prompt_suffix="<end_of_turn>\n<start_of_turn>model\n",
)
bundler.create_bundle(config)

print(f"模型 .task 包已保存到 {config.output_filename}")

## 下载并在设备上运行模型

你的模型现在已准备好用于通过 MediaPipe LLM 推理 API 在设备上运行推理！

从 Colab 环境下载 .task 文件以在你的项目中使用。

In [ ]:
from google.colab import files

files.download(config.output_filename)

你可以在[表情符号生成网页应用](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-mediapipe)中试用它，该应用直接在浏览器中运行模型。你也可以查看[文档](https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference)以构建跨平台移动和网页应用。